In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("piratemarket")

create_database('blp_dataset')
create_table('products_tmp_pm')

In [ ]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}")

    for file in os.listdir(folder):            
        if "index.php_s=items&id=" in file:
            try:
                page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())
            except:
                page_content = None

            print(f"{folder}\{file}")

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                
                desc_content = page_content.findAll("div", attrs={"class": "form-add"})

                if desc_content is not None and len(desc_content) > 0:
                    try:
                        desc_div = desc_content[1]
                    except:
                        desc_div = ""
                else:
                    desc_div = page_content.find("div", attrs={"class": "buy_item_description"})
                    
                if desc_div and desc_div is not None:
                    description = get_content(desc_div)

                ## GET TITLE
                title = ""
                raw_title = page_content.find("div", attrs={"class": "product-name"})

                if raw_title is None:
                    raw_title = page_content.find("div", attrs={"class": "buy_item_name"})
                    
                if raw_title is not None:
                    title = get_content(raw_title)

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_pm (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
                    values = (title, description, "Pirate Market", "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    main_div = page_content.find("p", attrs={"class": "product-image"})
                    
                    if main_div is not None:
                        img_content = main_div.find("img")
                        
                        if img_content is not None:
                            img_src = img_content["src"]

                            if img_src is not None:
                                url_image = Path(f"{folder}\{img_src}")

                                try:
                                    # copy images to new directory, renaming with product id
                                    if url_image.exists():
                                        print(f"Image {product_id} is normal")

                                        dst_dir = Path(f"D:\images\_pm\{product_id}.jpeg")
                                        copyfile(url_image, dst_dir)
                                    else:
                                        print(f"Image {product_id} does not exists")
                                except:
                                    print(f"Image {product_id} has error. Except")
                    else:
                        img_content = main_div = page_content.find("img")
                        
                        if img_content is not None:
                            img = img_content["src"]
                            
                            if img is not None:
                                try:
                                    data = "" + img.replace("data:image/jpg;base64,", "")
                                    data = "" + data.replace("data:image/jpeg;base64,", "")
                                    data = "" + data.replace("data:image/png;base64,", "")
                                    imgdata = base64.b64decode(data)

                                    img_filename = Path(f"D:\images\_pm\{product_id}.jpeg")

                                    with open(img_filename, 'wb') as f:
                                        f.write(imgdata)
                                        f.close
                                except:
                                    print(f"Image {product_id} has error. Except")